In [1]:
# Setting up
!pip install transformers
!pip install transformers[sentencepiece]

     |████████████████████████████████| 2.8 MB 5.3 MB/s 
     |████████████████████████████████| 52 kB 826 kB/s 
     |████████████████████████████████| 636 kB 47.2 MB/s 
     |████████████████████████████████| 895 kB 46.7 MB/s 
     |████████████████████████████████| 3.3 MB 40.5 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
     |████████████████████████████████| 1.1 MB 5.4 MB/s 


이전 챕터(2)의 예시로 이어서 살펴보자. sequence classifier를 하나의 batch로 학습시키는 것은 아래와 같다.

In [2]:
import torch
from transformers import AdamW, AutoTokenizer, AutoModelForSequenceClassification

checkpoint = 'bert-base-uncased'
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForSequenceClassification.from_pretrained(checkpoint)

sequences = [
    "I've been waiting for a HuggingFace course my whole life.",
    "This course is amazing!",
]

batch = tokenizer(sequences, padding=True, truncation=True, return_tensors='pt')

### 여기까지 이전과 같음

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [3]:
### 여기서부터 새로운 코드

batch['labels'] = torch.tensor([1, 1])

optimizer = AdamW(model.parameters())
loss = model(**batch).loss
loss.backward()
optimizer.step()

당연히 두 개의 문장만으로 training 시키는 거는 좋은 결과를 낼 수 없음. -> 훨씬 더 큰 dataset이 필요함.

### Loading a dataset from the Hub
- 데이터셋은 여기서 찾아볼 수 있음 https://huggingface.co/datasets
- 이번에는 MRPC dataset을 사용해 볼 것
- MRPC dataset은 GLUE benchmark를 구성하는 10개의 dataset 중 하나.

In [8]:
pip install datasets

     |████████████████████████████████| 270 kB 5.1 MB/s 
     |████████████████████████████████| 123 kB 64.9 MB/s 
     |████████████████████████████████| 1.3 MB 56.7 MB/s 
     |████████████████████████████████| 243 kB 64.9 MB/s 
     |████████████████████████████████| 294 kB 70.9 MB/s 
     |████████████████████████████████| 142 kB 67.2 MB/s 


In [9]:
from datasets import load_dataset

raw_datasets = load_dataset("glue", "mrpc")
raw_datasets

Downloading:   0%|          | 0.00/7.78k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/4.47k [00:00<?, ?B/s]

  0%|          | 0/3 [00:00<?, ?it/s]

Downloading: 0.00B [00:00, ?B/s]

Downloading: 0.00B [00:00, ?B/s]

...

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

Dataset glue downloaded and prepared to /root/.cache/huggingface/datasets/glue/mrpc/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 3668
    })
    validation: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 408
    })
    test: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 1725
    })
})

In [10]:
# dictionary니까 key를 이용해 접근 가능 
raw_train_dataset = raw_datasets["train"]
raw_train_dataset[0]

{'idx': 0,
 'label': 1,
 'sentence1': 'Amrozi accused his brother , whom he called " the witness " , of deliberately distorting his evidence .',
 'sentence2': 'Referring to him as only " the witness " , Amrozi accused his brother of deliberately distorting his evidence .'}

이미 label이 정수이므로 preprocessing 해줄 필요 없음. 어떤 정수가 어떤 label을 의미하는지 알아보자.

In [11]:
raw_train_dataset.features # 0은 not_equivalent, 1은 equivalent를 의미하는 것을 알 수 있다.

{'idx': Value(dtype='int32', id=None),
 'label': ClassLabel(num_classes=2, names=['not_equivalent', 'equivalent'], names_file=None, id=None),
 'sentence1': Value(dtype='string', id=None),
 'sentence2': Value(dtype='string', id=None)}

### Preprocessing pairs of sentences
이전 챕터에서는 단일 문장을 tokenizer에 feed 하거나 여러 문장으로 이루어진 list를 tokenizer에 feed 했음. 그럼 이와 비슷하게 모든 첫 번째 문장들과 모든 두 번째 문장들을 각각 tokenizer에 넣어주면 되지 않을까?

In [12]:
from transformers import AutoTokenizer

checkpoint = 'bert-base-uncased'
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

tokenized_sentences_1 = tokenizer(raw_datasets['train']['sentence1'])
tokenized_sentences_2 = tokenizer(raw_datasets['train']['sentence2'])

## 이렇게 하면 두 개의 sequence를 모델에 전달해서 하나의 예측을 수행할 수 없음.

다행히 tokenizer는 한 쌍의 sequence를 BERT 모델이 기대하는 대로 만들어줄 수 있음!

In [13]:
inputs = tokenizer("This is the first sentence.", "This is the second one.")
inputs

{'input_ids': [101, 2023, 2003, 1996, 2034, 6251, 1012, 102, 2023, 2003, 1996, 2117, 2028, 1012, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [15]:
print(tokenizer.convert_ids_to_tokens(inputs['input_ids']))
print(inputs['token_type_ids'])

# [CLS] sentence1 [SEP] -> token type ID: 0
# sentence2 [SEP] -> token type ID: 1

['[CLS]', 'this', 'is', 'the', 'first', 'sentence', '.', '[SEP]', 'this', 'is', 'the', 'second', 'one', '.', '[SEP]']
[0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1]


tokenizer가 어떻게 한 쌍의 sequence를 처리하는지 봤으니 전체 dataset을 tokenize 해보자.

In [16]:
tokenized_dataset = tokenizer(
    raw_datasets["train"]["sentence1"],
    raw_datasets["train"]["sentence2"],
    padding=True,
    truncation=True,
)

# 이렇게 tokenizer에 첫 번째 문장들 list와 두 번째 문장들 list을 한 쌍으로 줘도 되지만, (padding, truncation 옵션도 가능)
# dictionary(keys(input_ids, attention_mask, token_type_ids)와 values를 갖는)를 return한다는 단점이 있음.
# RAM이 충분한 경우에만 작동함. (huggingface dataset 라이브러리의 datasets가 디스크에 저장된 Apache Arrow 파일이기 때문에 사용자가 요청한 samples만 메모리에 올린 상태로 유지)


(위의 단점을 해결하기 위해) data를 dataset으로 유지하기 위해 Dataset.map method를 사용한다.

- tokenization 이상의 preprocessing이 필요한 경우에도 유연성을 준다.
- map method는 dataset의 각 요소에 함수를 적용함으로써 작동한다.

그러니까 input을 tokenize하는 함수를 정의해보자.

In [17]:
def tokenize_function(example):
    return tokenizer(example["sentence1"], example["sentence2"], truncation=True)

- 위 함수는 dataset의 items와 같은 하나의 dictionary를 받아서 새로운 dictionary를 return한다. (새로운 dictionary는 input_ids, attention_mask, token_type_ids를 key로 가짐)
- example dictionary가 여러 samples를 갖고 있어도 잘 작동한다 -> map 함수를 호출할 때 batched=True 옵션을 사용하게 해줌. batched=True 옵션은 tokenization 속도를 굉장히 높여 준다.
- padding을 아직 적용하지 않았음에 유의하자. -> 모든 samples에 대해 maximum length로 padding 하는 것은 효율적이지 않기 때문에, batch를 만들어줄 때 해당 batch에서 가장 긴 sentence 길이로 padding 하는 것이 좋다.

아래는 tokenization function을 모든 datasets에 한 번에 적용하는 방법이다. map을 호출할 때 batch=True를 사용했으니 tokenization function이 (각 요소에 개별적으로 적용되지 않고) dataset의 multiple elements에 한 번에 적용된다. 그래서 전처리가 더 빠르다.

In [18]:
tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)
tokenized_datasets

  0%|          | 0/4 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

DatasetDict({
    train: Dataset({
        features: ['attention_mask', 'idx', 'input_ids', 'label', 'sentence1', 'sentence2', 'token_type_ids'],
        num_rows: 3668
    })
    validation: Dataset({
        features: ['attention_mask', 'idx', 'input_ids', 'label', 'sentence1', 'sentence2', 'token_type_ids'],
        num_rows: 408
    })
    test: Dataset({
        features: ['attention_mask', 'idx', 'input_ids', 'label', 'sentence1', 'sentence2', 'token_type_ids'],
        num_rows: 1725
    })
})

### Dynamic padding
- collate function: 하나의 batch 내에 여러 sample들을 (여러 sample들 중에 이것 저것 선택해서 모아서..) 조립하는 함수.
- collate function은 Dataloader를 만들 때 사용하는 argument

HuggingFace Transformers 라이브러리는 DataCollatorWithPadding으로 그런 함수를 제공해준다.

In [19]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

예시로 살펴 보자. 8개의 문장 길이가 32에서 67로 다양하다.

In [20]:
samples = tokenized_datasets["train"][:8]
samples = {
    k: v for k, v in samples.items() if k not in ["idx", "sentence1", "sentence2"]
} # idx, sentence1, sentence2는 필요 없으니까 제거 (나머지만 samples에 넣어줌)
[len(x) for x in samples["input_ids"]]

[50, 59, 47, 67, 59, 50, 62, 32]

dynamic padding은 이 batch에 있는 sample들은 모두 길이가 67(이 batch에서의 최대 길이)이 되도록 padding 한다. dynamic padding을 사용하지 않으면 전체 데이터셋의 최대 길이로 모든 sample들을 padding 했을 것이다.

In [21]:
batch = data_collator(samples)
{k: v.shape for k, v in batch.items()}

{'attention_mask': torch.Size([8, 67]),
 'input_ids': torch.Size([8, 67]),
 'labels': torch.Size([8]),
 'token_type_ids': torch.Size([8, 67])}